In [1]:
import pymupdf

print(pymupdf.__doc__)

PyMuPDF 1.26.3: Python bindings for the MuPDF 1.26.3 library (rebased implementation).
Python 3.13 running on win32 (64-bit).



# Problem
* Based on https://github.com/pymupdf/PyMuPDF/discussions/763
* Start with looking at PyMuPDF PDF engine and its capabilities
* No build in equation detection, especially problematic for text based equation

# Pipeline
1. Equation detection
    * (option) verify detection
2. Equation conversion latex equation
    * (option) verify conversion
3. Replacing original equation with latex equation


## Equation detection
### 1. heuristic based equation detection
In our case code examples are not a problem:

Yes, exactly!
In PDF, text is just text. The PDF specification contains nothing to sub-divide different kinds of text. Equations are also text and be coded in any font, can be italic, or normal, mono-spaced of proportional, serifed or sans-serifed.
Also note that the equation symbol appears in program code listings a lot - PyMuPDF.pdf is full of such examples.

So I would say, that you have to develop your own way of recognizing equations ... and whatever you will develop, may not work with the next PDF example.


In [1]:
# [INFO]: MOVED TO MAIN DOCUMENT PIPELINE, NO LONGER MAINTAINED
# all to code moved to main pipeline

# Research on regex patterns to find equation on page

In [139]:
snippet_1 = """
1. The capital requirement for earthquake risk shall be equal to the following:


ffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffi
*SCR* *earthquake* ¼ ðX *CorrEQ* ð *r,s* Þ � *SCR* ð *earthquake,r* Þ � *SCR* ð *earthquake,s* Þ Þ þ *SCR* [2] ð *earthquake,other* Þ


s


*CorrEQ* ð *r,s* Þ � *SCR* ð *earthquake,r* Þ � *SCR* ð *earthquake,s* Þ Þ þ *SCR* [2] ð *earthquake,other* Þ


ð *r,s* Þ


where:
"""
snippet_2 = """
following amount:


ffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffiffi
*L* ð *earthquake,r* Þ ¼ *Q* ð *earthquake,r* Þ � X *Corr* ð *earthquake,r,i,j* Þ � *WSI* ð *earthquake,r,i* Þ � *WSI* ð *earthquake,r,j* Þ


s


*Corr* ð *earthquake,r,i,j* Þ � *WSI* ð *earthquake,r,i* Þ � *WSI* ð *earthquake,r,j* Þ


ð *i,j* Þ


where:

"""

In [ ]:
import os
with open(os.path.join("data", "raw", "solvency-II-files", f"{file_name}.pdf"), 'rb') as f:
        original_doc = pymupdf.open(f) # detection on original pdf

        for page in [original_doc[272]]:
            for image in page.get_images(full=True):
                 xref = image[0]
                 image_data = original_doc.extract_image(xref)
                 with open(f"image-page-{page.number + 1}-{xref}.{image_data['ext']}", "wb") as img_file:
                    img_file.write(image_data["image"])
            print(page.get_images(full=True))
            print(page.get_xobjects())

            page_text = page.get_text()
            with open(f"text-page-{page.number + 1}.txt", "w", encoding="utf-8") as text_file:
                text_file.write(page_text)
            # print(page.get_image_info(0)[0])



[(556, 0, 1235, 165, 1, '', '', 'I0', 'CCITTFaxDecode', 0), (1721, 0, 39, 48, 1, '', '', 'I1', 'CCITTFaxDecode', 0), (1720, 0, 34, 64, 1, '', '', 'I2', 'CCITTFaxDecode', 0), (1719, 0, 31, 62, 1, '', '', 'I3', 'CCITTFaxDecode', 0), (1718, 0, 1155, 388, 1, '', '', 'I4', 'CCITTFaxDecode', 0), (1720, 0, 34, 64, 1, '', '', 'I5', 'CCITTFaxDecode', 0), (1719, 0, 31, 62, 1, '', '', 'I6', 'CCITTFaxDecode', 0), (557, 0, 1020, 267, 1, '', '', 'I7', 'CCITTFaxDecode', 0), (1720, 0, 34, 64, 1, '', '', 'I8', 'CCITTFaxDecode', 0), (1719, 0, 31, 62, 1, '', '', 'I9', 'CCITTFaxDecode', 0)]
[]
